In [1]:
from sageresearch.discontinuousgalerkin import basis
from sageresearch.discontinuousgalerkin import canonical_element
from sageresearch.discontinuousgalerkin import flux_functions

# Discontinuous Galerkin Formulation

## Flux Integral

The dg formulation involves a integral of the flux multiplied by the gradient of the test functions or basis 
$$
    \int_{K_i}{\underline{f}(q, x, t) (\underline{D\phi}_i(x))^T} \mathrm{d}x
$$
The flux function $\underline{f}$ is a function whose output is a matrix of shape (num_eqns, num_dims)
$\underline{D\phi}$ is the jacobian of the basis functions in vector form, the output is a matrix of shape (num_basis_cpts, num_dims), $\underline{D\phi}_{jk} = \phi^j_{i,x_k}$
The tranpose is taken so that the flux function in each dimension is multiplied by the derivative of the basis functions in that direction.

This integral can be converted to the canonical element $\mathcal{K}$ using the linear transformations $c_i(x)$ and $b_i(\xi)$

$$
    \int_{b_i(K)}{\underline{f}(q, x, t) (\underline{D\phi}_i(x))^T} \mathrm{d}x = 
    \int_{K}{\underline{f}(q, b_i(\xi), t) (\underline{D_x\phi}_i(b_i(\xi)))^T} |b_i'(xi)| \mathrm{d} \xi
$$

The jacobian of the basis function can be simplified as follows
$$
    D_x(\phi_i(x)) = D_x(\phi(c_i(x)) = D_{\xi}\phi(c_i(x)) D_x c_i(x) = \phi'(c_i(x)) c_i'(x)
$$

$$
    D_x(\phi_i(x))_{x=b_i(\xi)} = \phi'(c_i(b_i(\xi)) c_i'(b_i(\xi)) = \phi'(\xi) c_i'(b_i(\xi)
$$

$$
    \int_{K_i}{\underline{f}(q, x, t) (\underline{D\phi}_i(x))^T} \mathrm{d}x = \int_{K} \underline{f}(q, b_i(\xi), t) \underline{c}_i'(b_i(\xi))^T \underline{\phi}'(\xi)^T |b_i'(\xi)|\mathrm{d} \xi
$$

### 1D

In [2]:
phi = vector(basis.get_legendre_basis_1d(4))
xi = canonical_element.get_canonical_variables_1d()
x = canonical_element.get_mesh_variables_1d()
vertex_list = canonical_element.get_mesh_element_vertices_1d()
c_i = canonical_element.get_transformation_to_canonical_1d_vertex_list(vertex_list)
b_i = canonical_element.get_transformation_to_mesh_1d_vertex_list(vertex_list)
phi_i = phi(c_i(x)).function(x)
d_phi = jacobian(phi, xi)
d_phi_i = jacobian(phi_i, x)
d_c_i = jacobian(c_i, x)
d_b_i = jacobian(b_i, xi)

$ D_x(\phi_i(x)) = \phi'(c_i(x)) c_i'(x) $

In [3]:
(d_phi(c_i(x)) * d_c_i(x) - d_phi_i(x)).simplify_full()

[0]
[0]
[0]
[0]

$ D_x(\phi_i(x))_{x=b_i(\xi)} = \phi'(\xi) c_i'(b_i(\xi) $

In [4]:
(d_phi_i(b_i(xi)) - d_phi(xi) * d_c_i(b_i(xi))).simplify_full()

[0]
[0]
[0]
[0]

$ \int_{K_i}{\underline{f}(q, x, t) (\underline{D\phi}_i(x))^T} \mathrm{d}x = \int_{K} \underline{f}(q, b_i(\xi), t) \underline{c}_i'(b_i(\xi))^T \underline{\phi}'(\xi)^T \mathrm{d} \xi $

In [9]:
q = sin(x).function(x)
f = flux_functions.get_advection_1d(1)
t = flux_functions.get_t_variable()
mesh_integrand = f(q(x), x, t) * d_phi_i(x).transpose()
mesh_integral = [canonical_element.integrate_over_mesh_element_1d_vertex_list(i, vertex_list) for i in mesh_integrand]
mesh_integral = matrix(mesh_integral)
canonical_integrand = f(q(b_i(xi)), b_i(xi), t) * d_c_i(b_i(xi)).transpose() * d_phi(xi).transpose() * d_b_i(xi).det()
canonical_integral = [canonical_element.integrate_over_canonical_element_1d(i) for i in canonical_integrand]
canonical_integral = matrix(canonical_integral)
(mesh_integral - canonical_integral).simplify_full()

[0 0 0 0]

### 2D Rectangles

In [ ]:
phi = vector(basis.get_legendre_basis_2d_rectangle(3))
xi = canonical_element.get_canonical_variables_1d()
x = canonical_element.get_mesh_variables_1d()
vertex_list = canonical_element.get_mesh_element_vertices_1d()
c_i = canonical_element.get_transformation_to_canonical_1d_vertex_list(vertex_list)
b_i = canonical_element.get_transformation_to_mesh_1d_vertex_list(vertex_list)
phi_i = phi(c_i(x)).function(x)
d_phi = jacobian(phi, xi)
d_phi_i = jacobian(phi_i, x)
d_c_i = jacobian(c_i, x)
d_b_i = jacobian(b_i, xi)

## Numerical Flux Boundary Integral

The section we will look at the term
$$
    \int_{\partial K_i} \underline{\underline{f}}^* \underline{n} \underline{\phi}_i^T(x) ds
$$

This term is the integral over the surface of the element $K_i$ of the numerical flux dotted into the outward normal vector times the basis function on the boundary.
This integral is very different in different dimensions, that is in 1D this integral is just an evaluation at the boundary which is a single point, in 2D this is a line integral, and in 3D it will be a surface integral.

### 1D

In 1D, $K_i = [x_l, x_r]$, and the outward normal vector at $x_l$ is $n = [-1]$ and at $x_r$ is $n = [1]$.
Therefore the boundary integral in 1D is
$$
    \int_{\partial K_i} \underline{\underline{f}}^* \underline{n} \underline{\phi}_i^T(x) ds = \underline{\underline{f}}^*(x_r) \underline{\phi}_i^T(x_r) - \underline{\underline{f}}^*(x_l) \underline{\phi}_i^T(x_l) 
$$

This can easily be transformed to the canonical element $\mathcal{K} = [-1, 1]$, using the transformation $b_i(\xi):\mathcal{K} \to K_i$
$$
    \underline{\underline{f}}^*(b_i(1)) \underline{\phi}^T(1) - \underline{\underline{f}}^*(b_i(-1)) \underline{\phi}^T(-1)
$$

In [2]:
phi = vector(basis.get_legendre_basis_1d(4))
ce = canonical_element.Interval()
xi = ce.get_canonical_variables()
x = ce.get_mesh_variables()
vertex_list = ce.get_mesh_element_vertices()
x_l = vertex_list[0, 0]
x_r = vertex_list[1, 0]
c_i = ce.get_transformation_to_canonical(vertex_list)
b_i = ce.get_transformation_to_mesh(vertex_list)
phi_i = phi(c_i(x)).function(x)
q = sin(x).function(x)
f = flux_functions.get_advection_1d(1)
t = flux_functions.get_t_variable()
f_star_l = f(q(x_l), x_l, t)
f_star_r = f(q(x_r), x_r, t)
mesh_boundary_integral = f_star_r * phi_i(x_r) - f_star_l * phi_i(x_l)
f_star_m1 = f(q(b_i(-1)), b_i(-1), t)
f_star_p1 = f(q(b_i(1)), b_i(1), t)
canonical_boundary_integral = f_star_p1 * phi(1) - f_star_m1 * phi(-1)
diff = mesh_boundary_integral - canonical_boundary_integral
assert norm(diff) == 0
diff

(0, 0, 0, 0)

### 2D

In two dimensions, the boundary integral is now a line integral.
In order to properly define this integral we need to parameterize the line in 1d.
Let $\underline{r}(s)$ be the parameterization of the line $L$ for $s \in [s_1, s_2]$, then the line integral is defined as
$$
    \int_L h(\underline{x}) dl = \int_{t_1}^{t_2} h(\underline{r}(s)) ||\underline{r}'(s)|| ds
$$

The boundary integral for the discontinuous galerkin method can be written as the sum of the integral over the faces of the element.
In particular if $\mathcal{F}_i$ is the set of faces on the mesh element with parameterizations $r_f(s)$ on $s \in [-1, 1]$, then

$$
    \int_{\partial K_i} \underline{\underline{f}}^* \underline{n} \underline{\phi}_i^T(x) ds = \sum_{f \in \mathcal{F}_i} 
        \int_{-1}^{1}\underline{\underline{f}}^* \underline{n} \underline{\phi}_i^T(r_f(s)) ||r_f'(s)|| ds
$$

This can also be written as a sum over line integrals over the canonical element.
In this case we have parameterizations of the faces of the canonicale element, $r_f(t)$ for $f \in \mathcal{F}$ the set of faces of the canonical element.
Then we can understand the parameterization of the faces of the mesh element as $\underline{b}_i(r_f(s))$.
This handles the transformation to the canonical element and the parameterization of the face in one step.
Note that this is still using the outward normal facing vector of the mesh element.

$$
    \int_{\partial K_i} \underline{\underline{f}}^* \underline{n} \underline{\phi}_i^T(x) ds = \sum_{f \in \mathcal{F}} 
        \int_{-1}^{1}\underline{\underline{f}}^* \underline{n} \underline{\phi}^T(r_f(s)) ||\underline{b}_i'(r_f(s)) r_f'(s)|| ds
$$

### 2D Rectangle

The parameterizations of the faces of the mesh element are
$$
    r_l = [x_l, \frac{y_t + y_b}{2} + \frac{s}{2}(y_t - y_b)] \\
    r_r = [x_r, \frac{y_t + y_b}{2} + \frac{s}{2}(y_t - y_b)] \\
    r_b = [\frac{x_l + x_r}{2} + \frac{s}{2}(x_r - x_l), y_b] \\
    r_t = [\frac{x_l + x_r}{2} + \frac{s}{2}(x_r - x_l), y_t]
$$

and the parameterizations of the faces of the canonoical element are
$$
    r_l = [-1, s] \\
    r_r = [1, s] \\
    r_b = [s, -1] \\
    r_t = [s, 1]
$$

The normal vectors of the mesh and canonical elements are equal and are given by
$$
    n_l = [-1, 0] \\
    n_r = [1, 0] \\
    n_b = [0, -1] \\
    n_t = [0, 1]
$$

In [68]:
square = canonical_element.Square()
vertex_list = square.get_mesh_element_vertices()
phi = vector(basis.get_legendre_basis_2d_rectangle(3))
c_i = square.get_transformation_to_canonical(vertex_list)
b_i = square.get_transformation_to_mesh(vertex_list)
tuple_ = square.get_mesh_variables()
x = tuple_[0]
y = tuple_[1]
phi_i = phi(c_i(x, y)[0], c_i(x, y)[1]).function(x, y)
mesh_faces_parameterizations = square.get_parameterizations_of_mesh_element_faces()
canonical_faces_parameterizations = square.get_parameterizations_of_canonical_element_faces()
s = square.get_parameterization_variable()
mesh_normal_vectors = square.get_outward_normal_vectors_mesh_element() 
canonical_normal_vectors = square.get_outward_normal_vectors_canonical_element()

f = flux_functions.get_advection_2d([1, 1])
t = flux_functions.get_t_variable()
q = (sin(x) * sin(y)).function(x, y)
x_s = [r_f(s)[0].function(s) for r_f in mesh_faces_parameterizations]
y_s = [r_f(s)[1].function(s) for r_f in mesh_faces_parameterizations]
num_faces = len(x_s)
f_star_m = [f(q(x_s[i](s), y_s[i](s)), x_s[i](s), y_s[i](s), t).function(s) for i in range(num_faces)]
n = mesh_normal_vectors
r_f_p_norm = [r_f.diff(s).norm() for r_f in mesh_faces_parameterizations]
# \dintt{-1}{1}{f^* n \phi_i^T(r_f(s)) \norm{r_f'(s)}}{s}
left_face_mesh_integral = integrate(f_star_m[0](s) * n[0] * phi_i(x_s[0](s), y_s[0](s)) * r_f_p_norm[0](s), s, -1, 1)
right_face_mesh_integral = integrate(f_star_m[1](s) * n[1] * phi_i(x_s[1](s), y_s[1](s)) * r_f_p_norm[1](s), s, -1, 1)
bottom_face_mesh_integral = integrate(f_star_m[2](s) * n[2] * phi_i(x_s[2](s), y_s[2](s)) * r_f_p_norm[2](s), s, -1, 1)
top_face_mesh_integral = integrate(f_star_m[3](s) * n[3] * phi_i(x_s[3](s), y_s[3](s)) * r_f_p_norm[3](s), s, -1, 1)
mesh_integral = left_face_mesh_integral + right_face_mesh_integral + bottom_face_mesh_integral + top_face_mesh_integral

xi_s = [r_f(s)[0].function(s) for r_f in canonical_faces_parameterizations]
eta_s = [r_f(s)[1].function(s) for r_f in canonical_faces_parameterizations]
x_s = [b_i(xi_s[i](s), eta_s[i](s))[0].function(s) for i in range(num_faces)]
y_s = [b_i(xi_s[i](s), eta_s[i](s))[1].function(s) for i in range(num_faces)]
f_star_c = [f(q(x_s[i](s), y_s[i](s)), x_s[i](s), y_s[i](s), t).function(s) for i in range(num_faces)]

# f_star_m should be same as f_star_c
assert sum([norm(f_star_m[i](s) - f_star_c[i](s)) for i in range(num_faces)]) == 0

b_i_jacobian = square.transform_to_mesh_jacobian(vertex_list)
b_i_j_r_f_p_norm = b_i_j_r_f_p_norm = [(b_i_jacobian * r_f.diff(s)).norm() for r_f in canonical_faces_parameterizations]
# \dintt{-1}{1}{f^* n \phi^T(r_f(s)) \norm{b_i'(r_f(s)) r_f'(s)}}{s}
left_face_canonical_integral = integrate(f_star_c[0](s) * n[0] * phi(xi_s[0](s), eta_s[0](s)) * b_i_j_r_f_p_norm[0](s), s, -1, 1)
right_face_canonical_integral = integrate(f_star_c[1](s) * n[1] * phi(xi_s[1](s), eta_s[1](s)) * b_i_j_r_f_p_norm[1](s), s, -1, 1)
bottom_face_canonical_integral = integrate(f_star_c[2](s) * n[2] * phi(xi_s[2](s), eta_s[2](s)) * b_i_j_r_f_p_norm[2](s), s, -1, 1)
top_face_canonical_integral = integrate(f_star_c[3](s) * n[3] * phi(xi_s[3](s), eta_s[3](s)) * b_i_j_r_f_p_norm[3](s), s, -1, 1)
canonical_integral = left_face_canonical_integral + right_face_canonical_integral + bottom_face_canonical_integral + top_face_canonical_integral

assert left_face_mesh_integral == left_face_canonical_integral
assert right_face_mesh_integral == right_face_canonical_integral
assert bottom_face_mesh_integral == bottom_face_canonical_integral
assert top_face_mesh_integral == top_face_canonical_integral
assert mesh_integral == canonical_integral

### 2D Triangle

The vertices of the triangular element are $v_0 = (x_0, y_0), v_1 = (x_1, y_1), v_2 = (x_2, y_2)$.
The parameterizations of the faces of the mesh element are
$$
        r_l = ((v_2 - v_1) * s + v_1 + v_2) / 2 \\
        r_b = ((v_3 - v_2) * s + v_2 + v_3) / 2 \\
        r_h = ((v_1 - v_3) * s + v_3 + v_1) / 2
$$

and the parameterizations of the faces of the canonoical element are
$$
    r_l = [-1, s] \\
    r_b = [s, -1] \\
    r_h = [s, -s]
$$

The normal vectors of the mesh elements are given by
$$
        n_l = [y_1 - y_0, x_0 - x_1] \\
        n_b = [y_2 - y_1, x_1 - x_2] \\
        n_h = [y_0 - y_2, x_2 - x_0]
$$
These are not unit length and need to be normalized as well.

In [11]:
triangle = canonical_element.Triangle()
vertex_list = triangle.get_mesh_element_vertices()
phi = vector(basis.get_modal_basis_2d_triangle(1))
c_i = triangle.get_transformation_to_canonical(vertex_list)
b_i = triangle.get_transformation_to_mesh(vertex_list)
tuple_ = triangle.get_mesh_variables()
x = tuple_[0]
y = tuple_[1]
phi_i = phi(c_i(x, y)[0], c_i(x, y)[1]).function(x, y)
mesh_faces_parameterizations = triangle.get_parameterizations_of_mesh_element_faces()
canonical_faces_parameterizations = triangle.get_parameterizations_of_canonical_element_faces()
s = triangle.get_parameterization_variable()
mesh_normal_vectors = triangle.get_outward_normal_vectors_mesh_element() 
canonical_normal_vectors = triangle.get_outward_normal_vectors_canonical_element()

In [12]:
f = flux_functions.get_advection_2d([1, 1])
t = flux_functions.get_t_variable()
q = (sin(x) * sin(y)).function(x, y)
x_s = [r_f(s)[0].function(s) for r_f in mesh_faces_parameterizations]
y_s = [r_f(s)[1].function(s) for r_f in mesh_faces_parameterizations]
num_faces = len(x_s)
f_star_m = [f(q(x_s[i](s), y_s[i](s)), x_s[i](s), y_s[i](s), t).function(s) for i in range(num_faces)]
n = mesh_normal_vectors
r_f_p_norm = [r_f.diff(s).norm() for r_f in mesh_faces_parameterizations]

In [15]:
left_face_mesh_integral = integrate(f_star_m[0](s) * n[0] * phi_i(x_s[0](s), y_s[0](s)) * r_f_p_norm[0](s), s, -1, 1)
bottom_face_mesh_integral = integrate(f_star_m[1](s) * n[1] * phi_i(x_s[1](s), y_s[1](s)) * r_f_p_norm[1](s), s, -1, 1)
hypotonuse_face_mesh_integral = integrate(f_star_m[2](s) * n[2] * phi_i(x_s[2](s), y_s[2](s)) * r_f_p_norm[2](s), s, -1, 1)
mesh_integral = left_face_mesh_integral + bottom_face_mesh_integral + hypotonuse_face_mesh_integral

In [16]:
triangle = canonical_element.Triangle()
vertex_list = triangle.get_mesh_element_vertices()
phi = vector(basis.get_modal_basis_2d_triangle(1))
c_i = triangle.get_transformation_to_canonical(vertex_list)
b_i = triangle.get_transformation_to_mesh(vertex_list)
tuple_ = triangle.get_mesh_variables()
x = tuple_[0]
y = tuple_[1]
phi_i = phi(c_i(x, y)[0], c_i(x, y)[1]).function(x, y)
mesh_faces_parameterizations = triangle.get_parameterizations_of_mesh_element_faces()
canonical_faces_parameterizations = triangle.get_parameterizations_of_canonical_element_faces()
s = triangle.get_parameterization_variable()
mesh_normal_vectors = triangle.get_outward_normal_vectors_mesh_element() 
canonical_normal_vectors = triangle.get_outward_normal_vectors_canonical_element()

f = flux_functions.get_advection_2d([1, 1])
t = flux_functions.get_t_variable()
q = (sin(x) * sin(y)).function(x, y)
x_s = [r_f(s)[0].function(s) for r_f in mesh_faces_parameterizations]
y_s = [r_f(s)[1].function(s) for r_f in mesh_faces_parameterizations]
num_faces = len(x_s)
f_star_m = [f(q(x_s[i](s), y_s[i](s)), x_s[i](s), y_s[i](s), t).function(s) for i in range(num_faces)]
n = mesh_normal_vectors
r_f_p_norm = [r_f.diff(s).norm() for r_f in mesh_faces_parameterizations]
# \dintt{-1}{1}{f^* n \phi_i^T(r_f(s)) \norm{r_f'(s)}}{s}
left_face_mesh_integral = integrate(f_star_m[0](s) * n[0] * phi_i(x_s[0](s), y_s[0](s)) * r_f_p_norm[0](s), s, -1, 1)
bottom_face_mesh_integral = integrate(f_star_m[1](s) * n[1] * phi_i(x_s[1](s), y_s[1](s)) * r_f_p_norm[1](s), s, -1, 1)
hypotonuse_face_mesh_integral = integrate(f_star_m[2](s) * n[2] * phi_i(x_s[2](s), y_s[2](s)) * r_f_p_norm[2](s), s, -1, 1)
mesh_integral = left_face_mesh_integral + bottom_face_mesh_integral + hypotonuse_face_mesh_integral

xi_s = [r_f(s)[0].function(s) for r_f in canonical_faces_parameterizations]
eta_s = [r_f(s)[1].function(s) for r_f in canonical_faces_parameterizations]
x_s = [b_i(xi_s[i](s), eta_s[i](s))[0].function(s) for i in range(num_faces)]
y_s = [b_i(xi_s[i](s), eta_s[i](s))[1].function(s) for i in range(num_faces)]
f_star_c = [f(q(x_s[i](s), y_s[i](s)), x_s[i](s), y_s[i](s), t).function(s) for i in range(num_faces)]

# f_star_m should be same as f_star_c
assert sum([norm(f_star_m[i](s) - f_star_c[i](s)) for i in range(num_faces)]) == 0

b_i_jacobian = square.transform_to_mesh_jacobian(vertex_list)
b_i_j_r_f_p_norm = b_i_j_r_f_p_norm = [(b_i_jacobian * r_f.diff(s)).norm() for r_f in canonical_faces_parameterizations]
# \dintt{-1}{1}{f^* n \phi^T(r_f(s)) \norm{b_i'(r_f(s)) r_f'(s)}}{s}
left_face_canonical_integral = integrate(f_star_c[0](s) * n[0] * phi(xi_s[0](s), eta_s[0](s)) * b_i_j_r_f_p_norm[0](s), s, -1, 1)
bottom_face_canonical_integral = integrate(f_star_c[1](s) * n[1] * phi(xi_s[1](s), eta_s[1](s)) * b_i_j_r_f_p_norm[1](s), s, -1, 1)
hypotonuse_face_canonical_integral = integrate(f_star_c[2](s) * n[2] * phi(xi_s[2](s), eta_s[2](s)) * b_i_j_r_f_p_norm[2](s), s, -1, 1)
canonical_integral = left_face_canonical_integral + bottom_face_canonical_integral + hypotonuse_face_canonical_integral

assert left_face_mesh_integral == left_face_canonical_integral
assert bottom_face_mesh_integral == bottom_face_canonical_integral
assert hypotonuse_face_mesh_integral == hypotonuse_face_canonical_integral
assert mesh_integral == canonical_integral

AssertionError: 